# Upstream Network Analysis

This notebook demonstrates how to analyze HMS basin network topology and calculate contributing drainage areas using the new upstream network analysis primitives.

**New in hms-commander**: These primitives were extracted from the DSS ReLink workflow to provide generalizable network analysis capabilities.

## What You'll Learn

1. Extracting diversion elements (CRITICAL for accurate network topology)
2. Building reverse network lookup (upstream relationships)
3. Finding all upstream elements for any target
4. Calculating total contributing drainage area
5. Extracting geometry with diversions for spatial verification

## Why This Matters

**Without diversions**: Upstream area calculations can be 8-10x wrong!

**Example (South Belt A1000000_1601_J)**:
- Without diversions: 6.04 sq mi (1 subbasin found)
- WITH diversions: 52.41 sq mi (14 subbasins found)
- **8.7x difference** due to missing diversion HICK0100_9901_D

## Prerequisites

- hms-commander installed with latest primitives
- An HMS basin model file (`.basin`)
- Optional: `.geo` and `.map` files for enhanced coordinates

**Estimated Total Runtime**: 2-5 minutes

In [ ]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

## Setup

In [ ]:
import sys
from pathlib import Path

# Ensure we use the local source code (not installed package)
# This is important during development when new features haven't been released yet
hms_commander_root = Path(r"C:\GH\hms-commander")
if str(hms_commander_root) not in sys.path:
    sys.path.insert(0, str(hms_commander_root))

import pandas as pd
import geopandas as gpd

from hms_commander import HmsBasin, HmsGeo, __version__

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(f"hms-commander v{__version__}")
print(f"Source: {Path(HmsBasin.__module__).parent if hasattr(HmsBasin, '__module__') else 'installed'}")

## Example 1: South Belt Stormwater Detention Basin (A100)

We'll start with the South Belt HMS model. This model has:
- 76 subbasins
- 47 junctions
- 13 diversions (CRITICAL - these split flows between multiple downstream paths)
- Complex network topology with branching and diversions

Later, we'll apply the same primitives to a second project (Meyergrove D100) to demonstrate generalizability.

**Note**: If you don't have these models, you can use any HMS basin file. The patterns work on any HMS project.

In [ ]:
# Path to HMS basin file
# Adjust this to your HMS project location
basin_path = Path(r"H:\25-001 HCFCD Benefits\Standard_Benefits_Process\3 - Model Data (CLB)\A520-03-00-E003 - South Belt Stormwater Detention Basin\A100_B100_HMS_413_A14_AllFreq_Optimized\Pre_1PCT.basin")

# Optional: geometry and map files for enhanced coordinates
geo_path = basin_path.parent / "A100-GEO.geo"
map_path = basin_path.parent / "A100-MAP.map"

print(f"Basin file: {basin_path.name}")
print(f"Exists: {basin_path.exists()}")

## 1. Extract Diversion Elements (CRITICAL)

**HmsGeo and previous HmsBasin methods completely ignored Diversion elements!**

Diversions are flow-splitting elements that route water to multiple downstream paths. Without them:
- Network topology is incomplete
- Upstream element discovery fails
- Drainage area calculations are catastrophically wrong (8-10x underestimation)

**New method**: `HmsBasin.get_diversions()` extracts diversions just like subbasins or junctions.

In [ ]:
# Extract diversion elements
diversions = HmsBasin.get_diversions(basin_path)

print(f"Found {len(diversions)} diversions\n")
diversions[['name', 'downstream', 'divert_to', 'canvas_x', 'canvas_y']].head(10)

**Key Observations**:
- Each diversion has TWO downstream paths: `downstream` and `divert_to`
- This splits flow between multiple locations
- Essential for complete network topology

## 2. Extract Other Element Types

For complete network analysis, we need all element types:

In [ ]:
# Extract all element types
subbasins = HmsBasin.get_subbasins(basin_path)
junctions = HmsBasin.get_junctions(basin_path)
reaches = HmsBasin.get_reaches(basin_path)

print(f"Subbasins: {len(subbasins)}")
print(f"Junctions: {len(junctions)}")
print(f"Reaches: {len(reaches)}")
print(f"Diversions: {len(diversions)}")
print(f"\nTotal elements: {len(subbasins) + len(junctions) + len(reaches) + len(diversions)}")

## 3. Build Reverse Network Lookup

HMS basin files define downstream connections ("what flows TO"). For upstream analysis, we need the reverse: "what flows FROM".

**New method**: `HmsBasin.get_upstream_network()` builds a reverse lookup dictionary:
- Key: Element name
- Value: List of elements that flow into it

**Includes diversions** - this is critical for complete topology!

In [ ]:
# Build reverse network lookup
upstream_network = HmsBasin.get_upstream_network(basin_path)

print(f"Total elements in network: {len(upstream_network)}")
print(f"\nSample - Elements flowing INTO A1000000_1601_J:")
if 'A1000000_1601_J' in upstream_network:
    upstream_of_target = upstream_network['A1000000_1601_J']
    print(f"  {len(upstream_of_target)} direct upstream elements:")
    for elem in upstream_of_target[:5]:
        print(f"    - {elem['name']} ({elem['type']})")
    if len(upstream_of_target) > 5:
        print(f"    ... and {len(upstream_of_target) - 5} more")

## 4. Find All Upstream Elements (Recursive)

To find EVERYTHING upstream of a target (not just direct connections), we need recursive traversal.

**New method**: `HmsBasin.get_upstream_elements()` recursively finds all upstream elements:
- Starts at target element
- Traverses network upstream
- Includes cycle detection (prevents infinite loops)
- Returns elements grouped by type

In [ ]:
# Find all upstream elements for a target junction
target = "A1000000_1601_J"

upstream = HmsBasin.get_upstream_elements(basin_path, target)

print(f"=== Upstream Elements for {target} ===")
print(f"Subbasins: {len(upstream['subbasins'])}")
print(f"Junctions: {len(upstream['junctions'])}")
print(f"Reaches: {len(upstream['reaches'])}")
print(f"Diversions: {len(upstream['diversions'])}")
print(f"\nTotal upstream elements: {sum(len(v) for v in upstream.values())}")

# Show the subbasins (these contribute drainage area)
print(f"\nUpstream subbasins (first 10):")
for sb in upstream['subbasins'][:10]:
    print(f"  - {sb}")
if len(upstream['subbasins']) > 10:
    print(f"  ... and {len(upstream['subbasins']) - 10} more")

**Critical Finding**: If diversions are missing, this method would find FAR FEWER upstream subbasins (1 instead of 14 for South Belt)!

## 5. Calculate Contributing Drainage Area

Once we have all upstream subbasins, we can sum their areas to get total contributing drainage area.

**New method**: `HmsBasin.get_contributing_area()` combines upstream discovery with area summation:
- Finds all upstream subbasins (including through diversions)
- Sums their drainage areas
- Returns total area in square miles

In [ ]:
# Calculate total contributing drainage area
area = HmsBasin.get_contributing_area(basin_path, target)

print(f"=== Contributing Drainage Area for {target} ===")
print(f"Total area: {area:.2f} sq mi")
print(f"\nThis includes flow from {len(upstream['subbasins'])} upstream subbasins")
print(f"routed through {len(upstream['diversions'])} diversions")

**South Belt Validation**:
- Expected: ~52.41 sq mi with diversions
- WITHOUT diversions: ~6.04 sq mi (8.7x underestimation!)

The difference is due to diversion HICK0100_9901_D routing flow from 13 additional subbasins.

## 6. Extract Geometry with Diversions

The existing `HmsGeo.extract_all_gis()` method now has enhanced support for diversions.

**New parameter**: `include_diversions=True` (default) creates `hms_diversions.geojson`

**Backward compatible**: Set `include_diversions=False` to skip diversions (legacy behavior)

In [ ]:
# Extract all geometry including diversions
output_dir = Path.cwd() / "upstream_analysis_outputs"
output_dir.mkdir(exist_ok=True)

outputs = HmsGeo.extract_all_gis(
    basin_path=basin_path,
    geo_path=geo_path if geo_path.exists() else None,
    map_path=map_path if map_path.exists() else None,
    output_dir=output_dir,
    crs_epsg="urn:ogc:def:crs:EPSG::2278",  # Texas State Plane Central (US Feet)
    include_diversions=True  # NEW: Include diversions in output
)

print("=== Extracted GeoJSON Files ===")
for key, path in outputs.items():
    if path.exists():
        gdf = gpd.read_file(path)
        print(f"{key:15} -> {path.name:30} ({len(gdf)} features)")

**Key Output**: `hms_diversions.geojson` is now available for spatial analysis and visualization!

## 7. Visualize Upstream Network

Let's create a simple map showing the target element and its upstream contributing subbasins.

In [ ]:
import matplotlib.pyplot as plt

# Read geometry
subbasins_gdf = gpd.read_file(outputs['subbasins'])
junctions_gdf = gpd.read_file(outputs['junctions'])
diversions_gdf = gpd.read_file(outputs['diversions'])

# Identify upstream subbasins
upstream_names = set(upstream['subbasins'])
subbasins_gdf['is_upstream'] = subbasins_gdf['name'].isin(upstream_names)

# Create figure
fig, ax = plt.subplots(figsize=(12, 10))

# Plot all subbasins (gray)
subbasins_gdf[~subbasins_gdf['is_upstream']].plot(
    ax=ax, color='lightgray', alpha=0.5, label='Other subbasins'
)

# Plot upstream subbasins (blue)
subbasins_gdf[subbasins_gdf['is_upstream']].plot(
    ax=ax, color='skyblue', alpha=0.7, edgecolor='blue', linewidth=1.5,
    label=f'Upstream subbasins ({len(upstream_names)})'
)

# Plot all junctions (small gray dots)
junctions_gdf.plot(ax=ax, color='gray', markersize=20, alpha=0.3, zorder=2)

# Plot diversions (red diamonds)
diversions_gdf.plot(
    ax=ax, color='red', marker='D', markersize=100, alpha=0.8, zorder=3,
    label=f'Diversions ({len(diversions_gdf)})'
)

# Highlight target junction (green star)
target_junction = junctions_gdf[junctions_gdf['name'] == target]
if not target_junction.empty:
    target_junction.plot(
        ax=ax, color='green', marker='*', markersize=500, zorder=4,
        label=f'Target: {target}'
    )

# Format
ax.set_title(
    f"Upstream Network Analysis: {target}\n"
    f"Contributing Area: {area:.2f} sq mi from {len(upstream_names)} subbasins",
    fontsize=14, fontweight='bold'
)
ax.set_xlabel('Easting (ft)', fontsize=12)
ax.set_ylabel('Northing (ft)', fontsize=12)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / f'upstream_network_{target}.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nMap saved to: {output_dir / f'upstream_network_{target}.png'}")

## 8. Analyze Multiple Targets

Let's calculate contributing areas for multiple junctions to see the range:

In [ ]:
# Get all junction names
all_junctions = junctions_gdf['name'].tolist()

# Calculate area for each junction (sample 10 random junctions)
import random
random.seed(42)
sample_junctions = random.sample(all_junctions, min(10, len(all_junctions)))

results = []
for junction in sample_junctions:
    area_sqmi = HmsBasin.get_contributing_area(basin_path, junction)
    upstream_elems = HmsBasin.get_upstream_elements(basin_path, junction)
    
    results.append({
        'Junction': junction,
        'Area_sqmi': area_sqmi,
        'Upstream_Subbasins': len(upstream_elems['subbasins']),
        'Upstream_Diversions': len(upstream_elems['diversions'])
    })

results_df = pd.DataFrame(results).sort_values('Area_sqmi', ascending=False)
print("=== Contributing Areas for Sample Junctions ===")
results_df

## Summary Statistics

In [ ]:
print("=== Summary Statistics ===")
print(f"Total junctions analyzed: {len(results_df)}")
print(f"\nContributing Area Range:")
print(f"  Min: {results_df['Area_sqmi'].min():.2f} sq mi")
print(f"  Max: {results_df['Area_sqmi'].max():.2f} sq mi")
print(f"  Mean: {results_df['Area_sqmi'].mean():.2f} sq mi")
print(f"  Median: {results_df['Area_sqmi'].median():.2f} sq mi")
print(f"\nUpstream Subbasin Count:")
print(f"  Min: {results_df['Upstream_Subbasins'].min()}")
print(f"  Max: {results_df['Upstream_Subbasins'].max()}")
print(f"  Mean: {results_df['Upstream_Subbasins'].mean():.1f}")
print(f"\nDiversions in Network:")
print(f"  Total diversions: {len(diversions)}")
print(f"  Junctions with diversions upstream: {(results_df['Upstream_Diversions'] > 0).sum()}")

## Example 2: Meyergrove Stormwater Detention Basin (D100)

The South Belt example above demonstrated the basics. Now let's apply the same primitives to a **larger, more complex** watershed to show they generalize.

**Meyergrove** (Brays Bayou tributary, D100 watershed):
- 66 subbasins (vs South Belt's 76)
- 5 diversions and 5 reservoirs
- 114.71 sq mi total watershed area
- HMS version 4.3

This model tests the primitives against different naming conventions, element types, and network complexity.

In [ ]:
# Example 2: Meyergrove (D100 Watershed)
meyer_basin = Path(r"H:\25-001 HCFCD Benefits\Standard_Benefits_Process\3 - Model Data\D500-11-00-E001 & D500-16-00-E001 - Meyergrove Stormwater Detention Basin-Detention\HMS Model (4.3)\JUL2020_D100.basin")

print(f"Basin file: {meyer_basin.name}")
print(f"Exists: {meyer_basin.exists()}")
print(f"Size: {meyer_basin.stat().st_size:,} bytes")

# Extract all element types
m_subs = HmsBasin.get_subbasins(meyer_basin)
m_juncs = HmsBasin.get_junctions(meyer_basin)
m_reaches = HmsBasin.get_reaches(meyer_basin)
m_divs = HmsBasin.get_diversions(meyer_basin)

print(f"\n=== Meyergrove D100 Element Counts ===")
print(f"Subbasins:  {len(m_subs)}")
print(f"Junctions:  {len(m_juncs)}")
print(f"Reaches:    {len(m_reaches)}")
print(f"Diversions: {len(m_divs)}")
print(f"Total:      {len(m_subs) + len(m_juncs) + len(m_reaches) + len(m_divs)}")

# Show diversions
print(f"\n=== Diversions ===")
m_divs[['name', 'downstream', 'divert_to']]

In [ ]:
# Build upstream network and find outlet
m_network = HmsBasin.get_upstream_network(meyer_basin)
print(f"Network entries: {len(m_network)}")

# Find outlet junction (no downstream)
m_outlets = []
for _, row in m_juncs.iterrows():
    if 'downstream' not in row or not row['downstream'] or row['downstream'] == '':
        m_outlets.append(row['name'])

print(f"Outlet junctions: {m_outlets}")

# Calculate contributing area for outlet
m_outlet = m_outlets[0] if m_outlets else list(m_network.keys())[0]
m_upstream = HmsBasin.get_upstream_elements(meyer_basin, m_outlet)
m_area = HmsBasin.get_contributing_area(meyer_basin, m_outlet)

print(f"\n=== Outlet Analysis: {m_outlet} ===")
print(f"Upstream subbasins:  {len(m_upstream['subbasins'])}")
print(f"Upstream junctions:  {len(m_upstream['junctions'])}")
print(f"Upstream diversions: {len(m_upstream['diversions'])}")
print(f"Contributing area:   {m_area:.2f} sq mi")
print(f"Total subbasin area: {m_subs['area'].sum():.2f} sq mi")
print(f"Delta:               {m_subs['area'].sum() - m_area:.2f} sq mi "
      f"({len(m_subs) - len(m_upstream['subbasins'])} subbasins not draining to outlet)")

In [ ]:
# Compare contributing areas across multiple junctions
m_sample_junctions = random.sample(m_juncs['name'].tolist(), min(15, len(m_juncs)))

m_results = []
for junc in m_sample_junctions:
    a = HmsBasin.get_contributing_area(meyer_basin, junc)
    u = HmsBasin.get_upstream_elements(meyer_basin, junc)
    m_results.append({
        'Junction': junc,
        'Area_sqmi': a,
        'Upstream_Subbasins': len(u['subbasins']),
        'Upstream_Diversions': len(u['diversions'])
    })

m_results_df = pd.DataFrame(m_results).sort_values('Area_sqmi', ascending=False)

print("=== Meyergrove D100: Contributing Areas for Sample Junctions ===")
print(f"  Area range: {m_results_df['Area_sqmi'].min():.2f} - {m_results_df['Area_sqmi'].max():.2f} sq mi")
print(f"  Subbasin count range: {m_results_df['Upstream_Subbasins'].min()} - {m_results_df['Upstream_Subbasins'].max()}")
print(f"  Junctions with diversions upstream: {(m_results_df['Upstream_Diversions'] > 0).sum()} of {len(m_results_df)}")
print()
m_results_df

## Comparison: South Belt vs Meyergrove

Two different watersheds, same primitives -- demonstrating generalizability.

In [ ]:
# Visualize Meyergrove upstream network (matching South Belt example)
m_output_dir = Path.cwd() / "upstream_analysis_outputs"

# Extract Meyergrove geometry
m_geo_path = meyer_basin.parent / "JUL2020_D100.geo"
m_map_path = meyer_basin.parent / "JUL2020_D100.map"

m_outputs = HmsGeo.extract_all_gis(
    basin_path=meyer_basin,
    geo_path=m_geo_path if m_geo_path.exists() else None,
    map_path=m_map_path if m_map_path.exists() else None,
    output_dir=m_output_dir / "meyergrove",
    crs_epsg="urn:ogc:def:crs:EPSG::2278",
    include_diversions=True
)

# Read geometry
m_subs_gdf = gpd.read_file(m_outputs['subbasins'])
m_juncs_gdf = gpd.read_file(m_outputs['junctions'])
m_divs_gdf = gpd.read_file(m_outputs['diversions'])

# Identify upstream subbasins of outlet
m_upstream_names = set(m_upstream['subbasins'])
m_subs_gdf['is_upstream'] = m_subs_gdf['name'].isin(m_upstream_names)

# Create side-by-side figure for both watersheds
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# --- South Belt (left) ---
subbasins_gdf[~subbasins_gdf['is_upstream']].plot(
    ax=ax1, color='lightgray', alpha=0.5)
subbasins_gdf[subbasins_gdf['is_upstream']].plot(
    ax=ax1, color='skyblue', alpha=0.7, edgecolor='blue', linewidth=1.5)
junctions_gdf.plot(ax=ax1, color='gray', markersize=10, alpha=0.3, zorder=2)
diversions_gdf.plot(ax=ax1, color='red', marker='D', markersize=80, alpha=0.8, zorder=3)
target_junction = junctions_gdf[junctions_gdf['name'] == target]
if not target_junction.empty:
    target_junction.plot(ax=ax1, color='green', marker='*', markersize=400, zorder=4)

ax1.set_title(
    f"South Belt (A100): {target}\n"
    f"{area:.2f} sq mi from {len(upstream['subbasins'])} subbasins, "
    f"{len(diversions)} diversions",
    fontsize=11, fontweight='bold')
ax1.set_xlabel('Easting (ft)')
ax1.set_ylabel('Northing (ft)')
ax1.grid(True, alpha=0.3)

# --- Meyergrove (right) ---
m_subs_gdf[~m_subs_gdf['is_upstream']].plot(
    ax=ax2, color='lightgray', alpha=0.5)
m_subs_gdf[m_subs_gdf['is_upstream']].plot(
    ax=ax2, color='lightsalmon', alpha=0.7, edgecolor='firebrick', linewidth=1.5)
m_juncs_gdf.plot(ax=ax2, color='gray', markersize=10, alpha=0.3, zorder=2)
m_divs_gdf.plot(ax=ax2, color='red', marker='D', markersize=80, alpha=0.8, zorder=3)
m_target_junc = m_juncs_gdf[m_juncs_gdf['name'] == m_outlet]
if not m_target_junc.empty:
    m_target_junc.plot(ax=ax2, color='green', marker='*', markersize=400, zorder=4)

ax2.set_title(
    f"Meyergrove (D100): {m_outlet}\n"
    f"{m_area:.2f} sq mi from {len(m_upstream['subbasins'])} subbasins, "
    f"{len(m_divs)} diversions",
    fontsize=11, fontweight='bold')
ax2.set_xlabel('Easting (ft)')
ax2.set_ylabel('Northing (ft)')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(m_output_dir / 'watershed_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Comparison map saved to: {m_output_dir / 'watershed_comparison.png'}")

In [ ]:
# Side-by-side comparison
comparison = pd.DataFrame({
    'Metric': [
        'Subbasins', 'Junctions', 'Reaches', 'Diversions',
        'Total Elements', 'Total Subbasin Area (sq mi)',
        'Outlet Contributing Area (sq mi)',
        'Outlet Upstream Subbasins'
    ],
    'South Belt (A100)': [
        len(subbasins), len(junctions), len(reaches), len(diversions),
        len(subbasins) + len(junctions) + len(reaches) + len(diversions),
        f"{subbasins['area'].sum():.2f}",
        f"{area:.2f}",
        len(upstream['subbasins'])
    ],
    'Meyergrove (D100)': [
        len(m_subs), len(m_juncs), len(m_reaches), len(m_divs),
        len(m_subs) + len(m_juncs) + len(m_reaches) + len(m_divs),
        f"{m_subs['area'].sum():.2f}",
        f"{m_area:.2f}",
        len(m_upstream['subbasins'])
    ]
})

print("=== Watershed Comparison ===")
comparison

## Key Takeaways

1. **Diversions are CRITICAL** -- Without them, network topology is incomplete and drainage areas are wrong
2. **get_diversions()** -- Extracts diversions just like subbasins (13 in South Belt, 5 in Meyergrove)
3. **get_upstream_network()** -- Builds reverse lookup for "what flows into each element"
4. **get_upstream_elements()** -- Recursively finds ALL upstream elements (includes cycle detection)
5. **get_contributing_area()** -- Sums drainage areas from all upstream subbasins
6. **extract_all_gis(include_diversions=True)** -- Creates `hms_diversions.geojson` for spatial analysis

## Demonstrated Generalizability

These primitives work across different HMS models:
- **South Belt (A100)**: 76 subbasins, 13 diversions, 52+ sq mi contributing area
- **Meyergrove (D100)**: 66 subbasins, 5 diversions, 114+ sq mi contributing area

## Use Cases

- **HMS-to-RAS Boundary Matching**: Verify spatial relationships between HMS outlets and RAS cross-sections
- **Drainage Area Validation**: Check if HMS subbasin areas match expected watershed areas
- **Network Topology QA**: Identify disconnected subbasins or missing connections
- **Flow Path Analysis**: Understand how water moves through the model
- **Model Documentation**: Generate maps and tables showing network structure

## References

- **Feature Dev Notes**: `C:\GH\hms-commander\feature_dev_notes\dss-relink-integration\`
- **DSS ReLink Workflow**: Complete HMS-to-RAS boundary matching example
- **South Belt Validation**: Real-world example with 94.4% match quality